In [3]:

import numpy as np
import pandas as pd

In [4]:
movies = pd.read_csv('D:/ML projects/Movie Recommender/tmdb_5000_movies.csv')
credits = pd.read_csv('D:\ML projects\Movie Recommender/tmdb_5000_credits.csv')

In [7]:
movies = movies.merge(credits,on='title') #merged movies and credits dataset on the basis of title 

In [9]:
#genre
#id for fetching movie posters
#keywords
#title
#overview : can be used for content filtering
#cast
#crew

movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [12]:
movies.isnull().sum() #to check if data is missing from any column of movies dataset

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [13]:
#Step 1 : Data Preprocessing
movies.dropna(inplace=True) #dropped movies which had data missing

In [14]:
movies.isnull().sum(
    
)

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [15]:
movies.duplicated().sum() #to check if any duplicate data is present

0

In [16]:
import ast
def convertToMeaningfulDict(obj): #convert genres list to a meaningful dict
    ListofId = []

    for i in ast.literal_eval(obj):
        ListofId.append(i['name'])
    return ListofId


In [17]:
movies['genres'] = movies['genres'].apply(convertToMeaningfulDict)

In [18]:
movies['keywords'] = movies['keywords'].apply(convertToMeaningfulDict)

In [20]:
def convert(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter +=1
        else:
            break
    return L

In [21]:
movies['cast'] = movies['cast'].apply(convert)

In [22]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [23]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [24]:
movies['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [25]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [26]:
movies['genres'] =movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])

In [28]:
movies['cast'] =movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies['crew'] =movies['crew'].apply(lambda x: [i.replace(" ","") for i in x])
movies['keywords'] =movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])

In [30]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [32]:
new_df = movies[['movie_id','title','tags']]

In [34]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

C:\Users\adarsh\AppData\Local\Temp\ipykernel_16556\1824047427.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))


In [36]:
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

C:\Users\adarsh\AppData\Local\Temp\ipykernel_16556\1380776331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())


In [37]:
new_df['tags'][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

In [38]:
#Step 2: text vectorization i.e converting text into vectors
#Techniques that can be used: Bag of Tags, Tf-idf , word2vec etc

In [39]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000 , stop_words='english')

In [40]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [41]:
vectors[0]


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [42]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [43]:
def stem(text):
    y = []

    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)

In [44]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\adarsh\AppData\Local\Temp\ipykernel_16556\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [45]:
new_df['tags'][0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [46]:
from sklearn.metrics.pairwise import cosine_similarity

In [47]:
similarity = cosine_similarity(vectors)

In [49]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x: x[1])[1:6]

[(539, 0.26089696604360174),
 (1194, 0.2581988897471611),
 (507, 0.25302403842552984),
 (260, 0.25110592822973776),
 (1216, 0.24944382578492943)]

In [50]:
def recommendMovies(movie):
    movie_index = new_df[new_df['title'] ==  movie].index[0] #for fetching index of movie according to the movie title entered by user
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x: x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [52]:
recommendMovies('Batman Begins')

The Dark Knight
The Dark Knight Rises
Batman
Batman & Robin
Batman


In [56]:
import pickle
pickle.dump(new_df.to_dict(),open('movies_dictionary.pkl','wb'))

In [57]:
pickle.dump(similarity,open('similarity.pkl','wb'))